##Import Libraries

In [58]:
import pandas as pd
import lightgbm as lgb
import math
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.neighbors import KNeighborsClassifier
from numpy.random import rand
from numpy.random import seed
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from scipy.stats import spearmanr
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from scipy import stats
import re
from functools import reduce
from sklearn.ensemble import RandomForestClassifier
from scipy.optimize import Bounds,  minimize
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
import seaborn as sns
import random
import time
import os
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_extraction.text import CountVectorizer
import requests
#from boruta import BorutaPy
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import SelectKBest
from sklearn.ensemble import ExtraTreesClassifier

## Import Dataset

In [20]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
df_row = pd.read_csv('/content/drive/My Drive/loan.csv')

In [22]:
df_row.shape

(2260668, 145)

#### Pre-Target Distribution

In [23]:
df_row['loan_status'].value_counts()

,count
loan_status,
Fully Paid,1041952
Current,919695
Charged Off,261655
Late (31-120 days),21897
In Grace Period,8952
Late (16-30 days),3737
Does not meet the credit policy. Status:Fully Paid,1988
Does not meet the credit policy. Status:Charged Off,761
Default,31


#### Cleaning the Dataset

In [24]:
df = df_row.loc[:, df_row.isnull().mean() < .8]
#df = df[(df.loan_status != 'In Grace Period')]
df = df.dropna(thresh=int(len(df.columns)*0.5))
colunas_objetos = df.columns[df.dtypes=='object']
df = df.apply(lambda x: x.fillna(0) if x.dtype.kind in 'biufc' else x.fillna('Sem_Info'))
df.shape

(2257886, 106)

In [25]:
df['loan_status'].value_counts()

,count
loan_status,
Fully Paid,1039844
Current,919695
Charged Off,261093
Late (31-120 days),21897
In Grace Period,8952
Late (16-30 days),3737
Does not meet the credit policy. Status:Fully Paid,1912
Does not meet the credit policy. Status:Charged Off,725
Default,31


### Target Definition

Tiramos da base todos que estão em periodo de carência (In grace period) porque n seria possível saber se seriam um bom ou mau pagador. Assumimos todos os que estão com contratos quitados, ou em aberto mas sem faturas vencidas como bons, e os demais clientes, como maus pagadores.

In [26]:
def get_target(target):
    if target == 'Fully Paid':
        return 0
    #elif target == 'Does not meet the credit policy. Status:Fully Paid':
    #    return 0
    elif target == 'Current':
        return 0
    else:
        return 1

In [27]:
df['target'] = df['loan_status'].apply(get_target)
df['target'].value_counts(normalize=True,dropna= False)

,proportion
target,
0,0.867864
1,0.132136


In [28]:
df.shape

(2257886, 107)

In [31]:
df[colunas_objetos] = df[colunas_objetos].apply(lambda x: x.astype('category').cat.codes)
target = 'target'
safra = 'issue_d'
variaveis = df.columns.drop([target,safra])

## Challenging: LGBM without Feature Selection

In [32]:
X_train, X_test,y_train, y__test= train_test_split(df[variaveis], df[target], test_size = 0.3)
clf = lgb.LGBMClassifier()
clf.fit(X_train, y_train)
predicao = clf.predict(X_test)
acuracia_total_FULL = accuracy_score(predicao, y__test)
acuracia_total_FULL

[LightGBM] [Info] Number of positive: 208820, number of negative: 1371700
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.106526 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 13143
[LightGBM] [Info] Number of data points in the train set: 1580520, number of used features: 104
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.132121 -> initscore=-1.882333
[LightGBM] [Info] Start training from score -1.882333


0.8969896658527295

In [ ]:
acuracia_total_FULL = np.round((acuracia_total_FULL*100),2)

## Challenging: LGBM with Feature Selection by Mutual Information Method

In [33]:
variaveis = df.columns.drop([target,safra])
X_train_mutual_info, X_test_mutual_info,y_train_mutual_info, y__test_mutual_info= train_test_split(df[variaveis], df[target].to_numpy(), test_size = 0.3)
mutual_info = mutual_info_classif(X_train_mutual_info, y_train_mutual_info)
mutual_info = pd.Series(mutual_info)
mutual_info.index = X_train_mutual_info.columns
lista_vp_filtrada_MI = mutual_info[mutual_info.sort_values(ascending=False)>0] #Ganho de Informação maior que zero

In [36]:
sel_cols = SelectKBest(mutual_info_classif, k=len(lista_vp_filtrada_MI))
sel_cols.fit(X_train_mutual_info, y_train_mutual_info)
X_filtrado = df[X_train_mutual_info.columns[sel_cols.get_support()]]
X_train, X_test,y_train, y_test_1= train_test_split(X_filtrado, df[target], test_size = 0.3)
clf = lgb.LGBMClassifier()
clf.fit(X_train, y_train)
predicao_1 = clf.predict(X_test)
acuracia_total_MI_model = accuracy_score(predicao_1, y_test_1)

[LightGBM] [Info] Number of positive: 208774, number of negative: 1371746
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.107512 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 12960
[LightGBM] [Info] Number of data points in the train set: 1580520, number of used features: 102
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.132092 -> initscore=-1.882587
[LightGBM] [Info] Start training from score -1.882587


In [ ]:
acuracia_total_MI_model = np.round((acuracia_total_MI_model*100),2)

In [37]:

print(len(lista_vp_filtrada_MI), "variaveis",  "; acuracia:", acuracia_total_MI_model)

103 variaveis ; acuracia: 0.9019852369324708


## Challenging: LGBM with Feature Selection by Extra Tress


In [38]:
def funcao_variaveis_importantes(DADOS,varesp = 'y'):
        DADOS = DADOS.drop([safra], axis=1)
        X = DADOS.loc [:, DADOS.columns != varesp]
        Y = DADOS.loc [:, DADOS.columns == varesp]
        warnings.filterwarnings("ignore")
        model = ExtraTreesClassifier()
        model.fit(X,Y)
        ind = X.shape[1]
        variaveis = X.columns[0:ind]
        importancia = pd.DataFrame(variaveis)
        features = pd.DataFrame(model.feature_importances_)
        importancia['Importancia']= features
        importancia = importancia.rename(columns = {0:"Variáveis","Importancia":"Importancia"})
        return importancia.sort_values(['Importancia'], ascending=False)

lista_vp_ExtraTrees = funcao_variaveis_importantes(df,varesp = 'target')
X_filtrado_ExtraTrees = df[lista_vp_ExtraTrees[lista_vp_ExtraTrees.Importancia>0]['Variáveis'].values] #Importância maior que zero
lista_vp_filtrada_ExtraTrees = X_filtrado_ExtraTrees.columns

In [39]:
X_train, X_test,y_train, y_test_2= train_test_split(X_filtrado_ExtraTrees, df[target], test_size = 0.3)
clf = lgb.LGBMClassifier()
clf.fit(X_train, y_train)
predicao_2 = clf.predict(X_test)
acuracia_ExtraTree_Model = accuracy_score(predicao_2, y_test_2)

[LightGBM] [Info] Number of positive: 208519, number of negative: 1372001
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.099528 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 13172
[LightGBM] [Info] Number of data points in the train set: 1580520, number of used features: 104
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.131931 -> initscore=-1.883995
[LightGBM] [Info] Start training from score -1.883995


In [40]:
print(len(lista_vp_filtrada_ExtraTrees), "variaveis",  "; acuracia:", acuracia_ExtraTree_Model)

104 variaveis ; acuracia: 0.9039896658527296


In [ ]:
acuracia_ExtraTree_Model = np.round((acuracia_ExtraTree_Modell*100),2)

## Challenging: LGBM with Feature Selection by Hybrid Method (Mutual Information e ExtraTrees)



In [41]:
df_combinado = df[lista_vp_filtrada_MI.index]
df_combinado['target'] = df.target
def funcao_variaveis_importantes(DADOS,varesp = 'y'):
        X = DADOS.loc [:, DADOS.columns != varesp]
        Y = DADOS.loc [:, DADOS.columns == varesp]
        warnings.filterwarnings("ignore")
        model = ExtraTreesClassifier()
        model.fit(X,Y)
        ind = X.shape[1]
        variaveis = X.columns[0:ind]
        importancia = pd.DataFrame(variaveis)
        features = pd.DataFrame(model.feature_importances_)
        importancia['Importancia']= features
        importancia = importancia.rename(columns = {0:"Variáveis","Importancia":"Importancia"})
        return importancia.sort_values(['Importancia'], ascending=False)

lista_vp_COMBINADO = funcao_variaveis_importantes(df_combinado,varesp = 'target')
X_filtrado_COMBINADO = df_combinado[lista_vp_COMBINADO[lista_vp_COMBINADO.Importancia>0]['Variáveis'].values] #Importância maior que zero
lista_vp_filtrada_COMBINADO = X_filtrado_COMBINADO.columns

In [42]:
X_train, X_test,y_train, y_test_3= train_test_split(X_filtrado_COMBINADO, df[target], test_size = 0.3)
clf = lgb.LGBMClassifier()
clf.fit(X_train, y_train)
predicao_3 = clf.predict(X_test)
acuracia_combinado_Model = accuracy_score(predicao_3, y_test_3)

[LightGBM] [Info] Number of positive: 209152, number of negative: 1371368
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.107667 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 12955
[LightGBM] [Info] Number of data points in the train set: 1580520, number of used features: 102
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.132331 -> initscore=-1.880503
[LightGBM] [Info] Start training from score -1.880503


In [44]:
print(len(lista_vp_filtrada_COMBINADO), "variaveis",  "; acuracia:", acuracia_combinado_Model)

102 variaveis ; acuracia: 0.9119940947729883


In [ ]:
acuracia_combinado_Model = np.round((acuracia_combinado_Model*100),2)

# Summary Final Model Performance

In [59]:
Metodo_de_Selecao = ['Sem Seleção','Informação Mutua', 'ExtraTrees','Combinado']
N_features = [df.shape[1]-2,len(lista_vp_filtrada_MI),len(lista_vp_filtrada_ExtraTrees),len(lista_vp_filtrada_COMBINADO)]
acuracia= [round(acuracia_total_FULL,3),round(acuracia_total_MI_model,3),round(acuracia_ExtraTree_Model,3),round(acuracia_combinado_Model,3)]
precisao = [(np.round(precision_score(predicao, y__test, average='weighted'),5)*100),
            (np.round(precision_score(predicao_1, y_test_1, average='weighted'),5)*100),
            (np.round(precision_score(predicao_2, y_test_2, average='weighted'),5)*100),
            (np.round(precision_score(predicao_3, y_test_3, average='weighted'),5)*100)]
recall = [(np.round(recall_score(predicao, y__test, average='macro'),5)*100),
            (np.round(recall_score(predicao_1, y_test_1, average='macro'),5)*100),
            (np.round(recall_score(predicao_2, y_test_2, average='macro'),5)*100),
            (np.round(recall_score(predicao_3, y_test_3, average='macro'),5)*100)]

F1_score_ = [(np.round(f1_score(predicao, y__test, average='weighted'),5)*100),
            (np.round(f1_score(predicao_1, y_test_1, average='weighted'),5)*100),
            (np.round(f1_score(predicao_2, y_test_2, average='weighted'),5)*100),
            (np.round(f1_score(predicao_3, y_test_3, average='weighted'),5)*100)]

In [86]:
resultados = pd.DataFrame({
    'Method Seleccion':Metodo_de_Selecao,
    'N° de Features':N_features,
    'Accuracy':acuracia,
    'F1 Score': F1_score_,
    'Precision':precisao,
    'Recall':recall})

resultados.sort_values(by=['Accuracy'], ascending=False)

,Method Seleccion,N° de Features,Accuracy,F1 Score,Precision,Recall
3,Combinado,102,91.2,90.94,90.80,91.13
2,ExtraTrees,104,90.4,90.60,90.10,90.65
1,Informação Mutua,103,90.2,88.10,89.79,91.00
0,Sem Seleção,105,89.7,84.58,88.89,90.91
